# Bem vindo ao EEG Stats!

**LEIA ATENTAMENTE ANTES DE COMEÇAR**


Certifique-se de seguir os seguintes passos:

1.   Este arquivo (EEG_Stats.ipynb) está salvo no seu google drive e você o abriu utilizando o Google Colab através do Google Drive.
2.   Crie uma pasta chamada "your_folders_here" em seu Google Drive.
3.   Dentro desta pasta que criou copie suas pastas contendo os arquivos Txts organizados conforme explicado abaixo.
4. Siga os procedimentos exatamente como indicado nos textos abaixo e só altere as células de código quando for pedido.




**Como organizar os arquivos TXTs:**

Se você está utilizando este script, provavelmente você possui um modelo experimental com duas variáveis independentes.
A nomeclatura de suas pastas deve seguir a seguinte regra:
1. A pasta de ter somente dois numeros em seu nome
2. O primeiro numero representa o número da primera variável independente
3. O segundo numero representa o número da segunda variável independente

Desta forma se você tiver 3 condições diferentes na primeira variável e 2 na segunda, você deve ter as seguintes pastas: 11, 12, 21, 22, 31, 32.


**Exemplo**

Digamos que seu desenho experimental consista em 3 tarefas diferentes, e dois momentos (pré e pós). 

A pasta "11" deverá conter seus arquivos txt referente a tarefa 1 momento pré;

A pasta "12" deverá conter seus arquivos txt referente a tarefa 1 momento pós;

A pasta "21" deverá conter seus arquivos txt referente a tarefa 2 momento pré;

A pasta "22" deverá conter seus arquivos txt referente a tarefa 2 momento pós;

A pasta "31" deverá conter seus arquivos txt referente a tarefa 3 momento pré;

A pasta "32" deverá conter seus arquivos txt referente a tarefa 3 momento pós;

Assumindo que esta estrutura de pastas e arquivos estão em seu Google Drive vamos seguir para o próximo passo.


**Instalação do Pandas Profiling**

Uma das bibliotecas que vamos utilizar não é padrão do Google Colab e precisamos fazer a instalação.
Para isso, execute a célula abaixo.
Ao final aparecerá um botão com a seguinte mensagem "RESTART RUNTIME"
Clique no botão e siga para a próxima etapa.

In [0]:
# instalando o pandas profiling
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

**Montagem do Google Drive**

Execute a célula abaixo. Para isso selecione a célula e clique no botão play no canto superior esquerdo da mesma, ou simplesmente a selecione e pressione CTRL+ENTER.

Irá aperecer uma mensagem com um link e um campo para digitar o código de autorização.
Clique no link e siga os passos do Google para autorizando o acesso ao seu Google Drive (não se preocupe, o acesso se encerra após a finalização do script).

Ao final do processo o Google disponibilizará uma série de caracteres. Clique no icone para copiar e os cole no campo do código de autorização da célula abaixo que você acabou de executar e pressione ENTER.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

**Importação das demais Bibliotecas**

Nesta etapa vamos importar as demais bibliotecas.
Basta executar a célula abaixo.

In [0]:
import os
import csv
import numpy as np
import pandas as pd
import scipy
#import scikit_posthocs as ph
import statsmodels
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from statsmodels.formula.api import ols
from tabulate import tabulate
from pandas_profiling import ProfileReport

**Define o Path para a pasta de trabalho no Google Drive**

Esta etapa é muito importante!
Clique na pasta na coluna a esquerda do colab para abrir a estrutra do seu google drive.
Navegue até a sua pasta "your_folders_here".
Selecione a pasta que contém a pasta "your_folders_here", clique nos 3 pontos a direita da mesma e selecione a opção "Copy path".

Agora altere a célula abaixo substituindo o texto *SUBSTITUA ESSE TEXTO* selcionando o mesmo e pressionando CRTL+V.

Sua célula deve conter somente a palavra *path* seguida do sinal de igual e com o valor colado entre aspas.

Ex: path = "/content/drive/My Drive/IPUB/EEG_Stats"

In [0]:
path = "SUBSTITUA ESSE TEXTO"

Verifica se os diretórios para salvar os resultados existem e os cria se necessário

In [0]:
'''Checks if results dir exists and creates it if necessary'''
if not os.path.exists(path + '/results'):
    os.makedirs(path + '/results')
if not os.path.exists(path + '/results/plots'):
    os.makedirs(path + '/results/plots')
if not os.path.exists(path + '/results/boxplots/'):
    os.makedirs(path + '/results/boxplots/')

Cria uma lista baseada nos diretórios dos arquivos TXTs

In [0]:
'''Creates a list of the usefull directories to concatenate'''
listdir = ([x[0] for x in os.walk(path + '/your_folders_here/')]) #create list with all sub-directories
listdir.remove(path + '/your_folders_here/') #remove sub-directory lib
listdir = [w.replace( (path + '/your_folders_here/'), '') for w in listdir]

Cria o arquivo da planilha geral

In [0]:
'''Creates the final sheet'''
outputfile = open((path + '/results/final_sheet.csv'), 'w')
writer = csv.writer(outputfile, delimiter=',',quotechar='"', quoting=csv.QUOTE_ALL)
#Adds the header
writer.writerow(['IndVar1','IndVar2','Fp1','Fp2','F7','F3','Fz','F4','F8','T3','C3',
            'Cz','C4','T4','T5','P3','Pz','P4','T6','O1','Oz','O2'])

Concatena os txts na planilha csv criada

In [0]:
#Concatenates each row of each txt file to the final_sheet.csv
for i in listdir:
    source = path + '/your_folders_here/' + i + '/'
    ind_var = i[0]
    dep_var = i[1]
    for txt_file in os.listdir(source):
        if txt_file.endswith('txt') == True:
            files_opened = open((source + txt_file))
            reader = csv.reader(files_opened, delimiter=' ')
            next(reader, None)  # skip the headers
            for row in reader:
                row = [n.replace('+', '') for n in row]
                writer.writerow([ind_var, dep_var] + row)
            files_opened.close() #closes txt files
outputfile.close() #closes the csv file
#prints end feedback
print("Final sheet done!")

Define a planilha criada como base de dados para a execução de todos os calculos.

In [0]:
data = pd.read_csv(path + '/results/final_sheet.csv', index_col=False) # Import final_sheet

Define as variáveis do desenho experimental utilizado

In [0]:
''' EXPERIMENT DESIGN INFORMATIONS '''
# Design Infos
bonferroni = 7 #Bonferroni correction factor
lista_eletrodos = ['Fp1','Fp2','F7','F3','Fz','F4','F8','T3','C3',
            'Cz','C4','T4','T5','P3','Pz','P4','T6','O1','Oz','O2']
condicao = data['IndVar1']
momento = data['IndVar2']
execucao_pre = (data.loc[data['IndVar1'] == 1]).loc[data['IndVar2'] == 1]
execucao_post = (data.loc[data['IndVar1'] == 1]).loc[data['IndVar2'] == 2]
execucao = (data.loc[data['IndVar1'] == 1])
observacao_pre = (data.loc[data['IndVar1'] == 2]).loc[data['IndVar2'] == 1]
observacao_post = (data.loc[data['IndVar1'] == 2]).loc[data['IndVar2'] == 2]
observacao = (data.loc[data['IndVar1'] == 2])
imaginacao_pre = (data.loc[data['IndVar1'] == 3]).loc[data['IndVar2'] == 1]
imaginacao_post = (data.loc[data['IndVar1'] == 3]).loc[data['IndVar2'] == 2]
imaginacao = (data.loc[data['IndVar1'] == 3])
pre_moment = (data.loc[data['IndVar2'] == 1])
post_moment = (data.loc[data['IndVar2'] == 2])
task_list = [execucao_pre, execucao_post, observacao_pre, observacao_post, imaginacao_pre, imaginacao_post]
task_list_pre = [execucao_pre, observacao_pre, imaginacao_pre]
task_list_post = [execucao_post, observacao_post, imaginacao_post]
tasks = [execucao, observacao, imaginacao]
moments = [pre_moment, post_moment]
condition_list = [execucao, observacao, imaginacao]
moment_list = [pre_moment, post_moment]

Define as variaveis envolvidas nos Plots

In [0]:
# Plot Styling
fase_array = ['Execution\nPre', 'Execution\nPost', 'Action\nObservation\nPre', 'Action\nObservation\nPost', 'Motor\nImagery\nPre', 'Motor\nImagery\nPost']
fase_tasks = ['Execution', 'Action\nObservation', 'Motor\nImagery']
fase_Moment = ['Pre','Post']
fmt_list = ['ks', 'ks', 'ko', 'ko', 'k^', 'k^']
ls_list = ['none', 'full', 'none', 'full', 'none', 'full']

task_array = ['Execution', 'Action\nObservation', 'Motor\nImagery']
fmt_list2 = ['ks', 'ko', 'k^']
ls_list2 = ['full', 'full', 'full']

moment_array = ['Pre', 'Post']
fmt_list3 = ['ks', 'ks']
ls_list3 = ['none', 'full']

frequency = 'Beta'

**Inicio da declaração das funções**

Define a função que remove outliers com Robust Sigma

In [0]:
def remove_outliers(data): # Removes outliers
    '''Creates the final sheet'''
    outputfile = open((path + '/results/final_sheet_wo_outliers.csv'), 'w')
    writer = csv.writer(outputfile, delimiter=',',quotechar='"',
                                            quoting=csv.QUOTE_ALL)
    #Adds the header
    writer.writerow(['IndVar1','IndVar2','Fp1','Fp2','F7','F3','Fz','F4','F8','T3','C3',
                'Cz','C4','T4','T5','P3','Pz','P4','T6','O1','Oz','O2'])

    def filter (fase):
        for x in lista_eletrodos:
            eletrodo = x
            i = fase[eletrodo]
            # Measure the percentile intervals and then estimate Standard Deviation of the distribution
            p90 = np.percentile(i, 90)
            p10 = np.percentile(i, 10)
            p50 = np.median(i)
            # Robust Sigma
            rSig = (p90-p50)/1.2815
            rSig = (p90-p10)/(2*1.2815)
            # Remove outliers
            filtered = i[abs(i - p50) < rSig*5]
            fase[eletrodo] = filtered
            #print (i.mean())
            i.fillna(i.mean(), inplace=True)
        return fase

    ind_var = 0
    dep_var = 0
    for fase in task_list:
        filter(fase)
        np.savetxt(outputfile, fase, delimiter=',', fmt='"%1.8f"')
    outputfile.close() #closes the csv file
    print ('Outliers removed!')

Define a função que executa uma two-way ANOVA e Teste-T quando necessário

In [0]:
def run_stats_twa_tt(data): # Runs Statistics Two-way Anova and T-test when needed
    # Variables
    eletrodo = ''
    plotly_form = ' ~ C(condicao) + C(momento) + C(condicao):C(momento)'
    sig_con = ''
    sig_mom = ''
    sig_conxmom = ''
    p = 0.05/bonferroni #Bonferroni Correction for 7 Hypothesis

    ''' Loop for Two-way ANOVA for each Electrode '''
    for x in lista_eletrodos:
        eletrodo = x
        # Begining of ANOVA
        formula = (eletrodo + plotly_form)
        model = ols(formula, data, missing='drop').fit()
        aov_table = statsmodels.stats.anova.anova_lm(model, typ=2)
        sig_con = aov_table['PR(>F)']['C(condicao)']
        sig_mom = aov_table['PR(>F)']['C(momento)']
        sig_conxmom = aov_table['PR(>F)']['C(condicao):C(momento)']
        outputfile.write('\n\n\n\n' + str(model.summary()))
        outputfile.write('\n\nTwo-way Anova do Eletrodo: %s \n' % eletrodo)
        outputfile.write(tabulate(aov_table, headers='keys'))
        outputfile.write('\n\n')

        if (sig_conxmom < p):

            ''' T-test and Post Hoc Scheffe (POST HOC ON TESTS)'''
            '''MODIFY ACCORDING TO DESIGN INFORMATIONS'''
            twosample_results = scipy.stats.ttest_ind(execucao_pre[eletrodo], execucao_post[eletrodo])
            outputfile.write ('The t-statistic for execucao is %.3f and the p-value is %.3f.\n' % twosample_results)

            twosample_results = scipy.stats.ttest_ind(observacao_pre[eletrodo],observacao_post[eletrodo])
            outputfile.write ('The t-statistic for observacao is %.3f and the p-value is %.3f.\n' % twosample_results)

            twosample_results = scipy.stats.ttest_ind(imaginacao_pre[eletrodo], imaginacao_post[eletrodo])
            outputfile.write ('The t-statistic for imaginacao is %.3f and the p-value is %.3f.\n' % twosample_results)

        elif (sig_conxmom >= p and sig_con < p and sig_mom < p):
            outputfile.write ('Main effect for condition and moment.\n')
            '''incluir PostHoc para condicao'''
            #print ('Post Hoc Scheffe: %s' %eletrodo)
            #dataExe = data.loc[data['IndVar1'] == 1]
            #print (dataExe)
            #print (dataExe[eletrodo])
            #ph.posthoc_scheffe(dataExe)
        elif (sig_conxmom >= p and sig_mom < p):
            outputfile.write ('Main effect for moment.\n')
        elif (sig_conxmom >= p and sig_con < p):
            outputfile.write ('Main effect for condition.\n')
            '''incluir PostHoc para condicao'''
        else:
            outputfile.write ('No interection and no main effects.\n')
    outputfile.write('\n\n')

    #prints end feedback
    print("Stats done!")

Define a função que executa uma Two-way ANOVA e uma One-way ANOVA quando necessário

In [0]:
def run_stats_twa_owa(data): # Runs Statistics Two-way Anova and One-way Anova when needed
    # Variables
    eletrodo = ''
    plotly_form = ' ~ C(tarefa) + C(grupo) + C(tarefa):C(grupo)'
    sig_con = ''
    sig_mom = ''
    sig_conxmom = ''
    p = 0.05/bonferroni #Bonferroni correction for 7 Hypothesis

    ''' Loop for Two-way ANOVA for each Electrode '''
    for x in lista_eletrodos:
        eletrodo = x
        # Begining of Two-way ANOVA
        formula = (eletrodo + plotly_form)
        model = ols(formula, data, missing='drop').fit()
        aov_table = statsmodels.stats.anova.anova_lm(model, typ=2)
        sig_con = aov_table['PR(>F)']['C(tarefa)']
        sig_mom = aov_table['PR(>F)']['C(grupo)']
        sig_conxmom = aov_table['PR(>F)']['C(tarefa):C(grupo)']
        outputfile.write('\n\nTwo-way Anova do Eletrodo: %s \n' % eletrodo)
        outputfile.write(tabulate(aov_table, headers='keys'))
        outputfile.write('\n\n')

        if (sig_conxmom < p):

            '''One-way ANOVA'''
            '''MODIFY ACCORDING TO DESIGN INFORMATIONS'''
            outputfile.write('One-way Anova:')

            Fa, pa = stats.f_oneway(execucao_Ctrl[eletrodo], observacao_Ctrl[eletrodo], imaginacao_Ctrl[eletrodo])
            outputfile.write('\nControl group: F statistic = {:5.3f} and probability p = {:5.3f}'.format(Fa, pa))

            Fa, pa = stats.f_oneway(execucao_Cimp[eletrodo], observacao_Cimp[eletrodo], imaginacao_Cimp[eletrodo])
            outputfile.write('\nGroup with electrode: F statistic = {:5.3f} and probability p = {:5.3f}'.format(Fa, pa))

            Fa, pa = stats.f_oneway(execucao_Simp[eletrodo], observacao_Simp[eletrodo], imaginacao_Simp[eletrodo])
            outputfile.write('\nGroup without electrode: F statistic = {:5.3f} and probability p = {:5.3f}'.format(Fa, pa))
            outputfile.write('\n ')

        elif (sig_conxmom >= p and sig_con < p and sig_mom < p):
            outputfile.write ('Main effect for task and group.\n')
        elif (sig_conxmom >= p and sig_mom < p):
            outputfile.write ('Main effect for group.\n')
        elif (sig_conxmom >= p and sig_con < p):
            outputfile.write ('Main effect for task.\n')
        else:
            outputfile.write ('No interection and no main effects.\n')
    outputfile.write('\n\n')

    #prints end feedback
    print("Stats done!")

Define a função que gera os Plots e os salva em Vetor (.eps) e Raster (.png)

In [0]:
def plot_to_files (data): # Generates Plots Vector and Raster for each electrode
    # Export path
    plot_directory = path + '/results/plots/'
    # Calc y axe limits
    ylim_min = 0
    ylim_max = 0
    for eletrodo in lista_eletrodos:
        i=0
        for fase in task_list:
            x = fase[eletrodo]
            means = x.mean(0)
            std = x.std(0)
            outputfile.write('Electrode:%s ' % eletrodo + fase_array[i])
            outputfile.write('\n')
            outputfile.write('Mean: %s\n' % means)
            outputfile.write('Standard Deviation: %s\n' % std)
            outputfile.write('\n')
            i += 1
            if ((means-std) < ylim_min):
                ylim_min = (means-std)
            elif ((means+std) > ylim_max):
                ylim_max = (means+std)

    for eletrodo in lista_eletrodos:
        for moment in moment_list:
            x = moment[eletrodo]
            means = x.mean(0)
            std = x.std(0)
            if ((means-std) < ylim_min):
                ylim_min = (means-std)
            elif ((means+std) > ylim_max):
                ylim_max = (means+std)

    for eletrodo in lista_eletrodos:
        for task in condition_list:
            x = task[eletrodo]
            means = x.mean(0)
            std = x.std(0)
            if ((means-std) < ylim_min):
                ylim_min = (means-std)
            elif ((means+std) > ylim_max):
                ylim_max = (means+std)


    # Interaction Plots
    for eletrodo in lista_eletrodos:
        count = 0
        x_value = 0.5
        inc = True
        for fase in task_list: # group results in pairs
            if (inc == False):
                x_value += 0.5
                inc = True
            else:
                x_value += 1
                inc = False
            count += 1
            fmt_stl = fmt_list[count-1]
            ls_stl = ls_list[count-1]
            x = fase[eletrodo]
            means = x.mean(0)
            std = x.std(0)
            plt.errorbar(x_value, means, std, fmt=fmt_stl, fillstyle=ls_stl, lw=1, capsize=4, ecolor='k', markeredgewidth=1, mec='k')

        # Axes and Labels
        plt.box(on=False)
        plt.xlim(0, (count+1))
        plt.xticks(np.arange(1.75, count, step=1.5), fase_tasks, rotation=0)
        plt.ylim((ylim_min-1), (ylim_max+1))
        plt.ylabel('Absolute %s Power (\u03BCV\u00b2)' %frequency)
        plt.title('Electrode %s' % eletrodo)
        plt.axhline(ylim_min - 1, color='k')
        plt.axvline(0, color='k')

        # Legend Box
        patches = []
        for (i, l) in zip(ls_list3, moment_array):
            if (i == 'full'):
                fcolor = 'black'
            else:
                fcolor = 'white'
            patches = patches + [mpatches.Patch(edgecolor='black', facecolor=fcolor, label=l)]

        plt.legend(handles=patches, title='Moment')

        # Save plot to file
        your_vector_file_name = ('PlotInteraction_%s.eps' %eletrodo)
        your_raster_file_name = ('PlotInteraction_%s.png' %eletrodo)
        save_vector_file = os.path.join(plot_directory, your_vector_file_name)
        save_raster_file = os.path.join(plot_directory, your_raster_file_name)
        plt.savefig(save_vector_file)
        plt.savefig(save_raster_file, dpi=300)
        plt.close()
    #prints end feedback
    print("All Interaction Plots saved!")

    # Condition Plots
    for eletrodo in lista_eletrodos:
        count = 0
        for task in condition_list:
            count += 1
            fmt_stl = fmt_list2[count-1]
            ls_stl = ls_list2[count-1]
            x = task[eletrodo]
            means = x.mean(0)
            std = x.std(0)
            plt.errorbar(count, means, std, fmt=fmt_stl, fillstyle=ls_stl, lw=1, capsize=4, ecolor='k', markeredgewidth=1, mec='k')

        # Axes and Labels
        plt.box(on=False)
        plt.xlim(0, (count + 1))
        plt.xticks(range(1, (count + 1)), task_array, rotation=0)
        plt.ylim((ylim_min - 1), (ylim_max + 1))
        plt.ylabel('Absolute %s Power (\u03BCV\u00b2)' %frequency)
        plt.title('Electrode %s' % eletrodo)
        plt.axhline(ylim_min - 1, color='k')
        plt.axvline(0, color='k')

        # Save plot to file
        your_vector_file_name = ('PlotCondition_%s.eps' %eletrodo)
        your_raster_file_name = ('PlotCondition_%s.png' %eletrodo)
        save_vector_file = os.path.join(plot_directory, your_vector_file_name)
        save_raster_file = os.path.join(plot_directory, your_raster_file_name)
        plt.savefig(save_vector_file)
        plt.savefig(save_raster_file, dpi=300)
        plt.close()
    #prints end feedback
    print("All Condition Plots saved!")

    # Moment Plots
    for eletrodo in lista_eletrodos:
        count = 0
        for moment in moment_list:
            count += 1
            fmt_stl = fmt_list3[count-1]
            ls_stl = ls_list3[count-1]
            x = moment[eletrodo]
            means = x.mean(0)
            std = x.std(0)
            plt.errorbar(count, means, std, fmt=fmt_stl, fillstyle=ls_stl, lw=1, capsize=4, ecolor='k', markeredgewidth=1, mec='k')

        # Axes and Labels
        plt.box(on=False)
        plt.xlim(0, (count+1))
        plt.xticks(range(1, (count+1)), moment_array, rotation=0)
        plt.ylim((ylim_min-1), (ylim_max+1))
        plt.ylabel('Absolute %s Power (\u03BCV\u00b2)' % frequency)
        plt.title('Electrode %s' % eletrodo)
        plt.axhline(ylim_min - 1, color='k')
        plt.axvline(0, color='k')

        # Save plot to file
        your_vector_file_name = ('PlotMoment_%s.eps' %eletrodo)
        your_raster_file_name = ('PlotMoment_%s.png' %eletrodo)
        save_vector_file = os.path.join(plot_directory, your_vector_file_name)
        save_raster_file = os.path.join(plot_directory, your_raster_file_name)
        plt.savefig(save_vector_file)
        plt.savefig(save_raster_file, dpi=300)
        plt.close()
    #prints end feedback
    print("All Moment Plots saved!")

    #Plot TaskMoment
    for eletrodo in lista_eletrodos:
        count = 0
        x_pre_values = [None]
        y_pre_values = [None]
        x_pos_values = [None]
        y_pos_values = [None]
        max_mean = 0
        min_mean = means

        for fase in task_list_pre:
            count += 1
            x_pre_values = np.append(x_pre_values, [count])
            x = fase[eletrodo]
            means = x.mean(0)
            y_pre_values = np.append(y_pre_values, [means])
            if (means > max_mean):
                max_mean = means
            elif (means < min_mean):
                min_mean = means

        count = 0

        for fase in task_list_post:
            count += 1
            x_pos_values = np.append(x_pos_values, [count])
            x = fase[eletrodo]
            means = x.mean(0)
            y_pos_values = np.append(y_pos_values, [means])
            if (means > max_mean):
                max_mean = means
            elif (means < min_mean):
                min_mean = means

        # Axes and Labels
        plt.xlim(0, (count+1))
        plt.xticks(range(1, (count+1)), fase_tasks, rotation=0)
        plt.ylim((ylim_min-1), (ylim_max+1))
        plt.ylabel('Estimated marginal means')
        plt.title(eletrodo)
        plt.plot(x_pre_values, y_pre_values, 'b', label='Pre')
        plt.plot(x_pos_values, y_pos_values, 'r', label='Post')
        plt.legend()

        # Save plot to file
        your_vector_file_name = ('PlotTaskMoment_%s.eps' %eletrodo)
        your_raster_file_name = ('PlotTaskMoment_%s.png' %eletrodo)
        save_vector_file = os.path.join(plot_directory, your_vector_file_name)
        save_raster_file = os.path.join(plot_directory, your_raster_file_name)
        plt.savefig(save_vector_file)
        plt.savefig(save_raster_file, dpi=300)
        plt.close()

    #prints end feedback
    print("All Task-Moment Plots saved!")
    print("All Plots saved!")

Define a função que gera os Box plots

In [0]:
def boxPlot(data):
    # Checks if results dir exists and creates it if necessary
    if not os.path.exists(path + '/results'):
        os.makedirs(path + '/results')
    if not os.path.exists(path + '/results/boxplots/'):
        os.makedirs(path + '/results/boxplots/')

    # Calc y axe limits
    ylim_min = 0
    ylim_max = 0
    for eletrodo in lista_eletrodos:
        for task in task_list:
            minvalue = np.amin(task[eletrodo])
            maxvalue = np.amax(task[eletrodo])
            if (minvalue < ylim_min):
                ylim_min = minvalue
            elif (maxvalue > ylim_max):
                ylim_max = maxvalue

    # Export path
    plot_directory = path + '/results/boxplots/'
    totalLen = len(fase_array)
    for eletrodo in lista_eletrodos:
        fig, ax1 = plt.subplots(figsize=(10, 6))
        #ax1.boxplot([experiente_pre[eletrodo], experiente_post[eletrodo], inexperiente_pre[eletrodo], inexperiente_post[eletrodo]])
        ax1.boxplot([task[eletrodo] for task in task_list], widths=0.1)

        # Begining Styling of Plot
        fig.canvas.set_window_title('Eletrodo: %s' %eletrodo)
        fig.subplots_adjust(left=0.075, right=0.95, top=0.9, bottom=0.25)

        # Add a horizontal grid to the plot, but make it very light in color
        # so we can use it for reading data values but not be distracting
        ax1.yaxis.grid(True, linestyle='-', which='major', color='lightgrey', alpha=0.5)

        # Hide these grid behind plot objects
        ax1.set_axisbelow(True)
        ax1.set_title('Electrode: %s' %eletrodo)
        ax1.set_xlabel('Task / Moment', labelpad=20)
        ax1.set_ylabel('Value', labelpad=10)

        # Set the axes ranges and axes labels
        ax1.set_xlim(0.5, totalLen + 0.5)
        ax1.set_ylim((ylim_min-1), (ylim_max+1))
        ax1.set_xticklabels(fase_array, fontsize=8, horizontalalignment='center')

        plt.box(on=False)
        plt.axhline((ylim_min-1), color='k')
        plt.axvline(.5, color='k')

        #plt.show()

        # Save plot to file
        your_vector_file_name = ('BoxPlot_%s.eps' %eletrodo)
        your_raster_file_name = ('BoxPlot_%s.png' %eletrodo)
        save_vector_file = os.path.join(plot_directory, your_vector_file_name)
        save_raster_file = os.path.join(plot_directory, your_raster_file_name)
        plt.savefig(save_vector_file)
        plt.savefig(save_raster_file, dpi=300)
        plt.close()
    print ('Boxplots done!')

Inicio da execução das funçoes

Cria um arquivo para com os resultados dos calculos

In [0]:
'''Creates the final report'''
outputfile = open((path + '/results/final_report.txt'), 'w')

Remove os outliers

In [0]:
remove_outliers(data)

Atualiza a base de dados para a planilha sem outliers

In [0]:
data = pd.read_csv(path + '/results/final_sheet_wo_outliers.csv', index_col=False) # Updates data to final sheet without outliers

## **Executar somente uma das celulas abaixo de ANOVA de acordo com seu desenho experimental**

Executar esta celula para desenhos experimentais que utilizam Two-way ANOVA + Teste-T 

In [0]:
run_stats_twa_tt(data) #Choose this function for two-way anova with t-test

Executar esta celula para desenhos experimentais que utilizam Two-way ANOVA + One-way ANOVA

In [0]:
run_stats_twa_owa(data) #Choose this function for two-way anova with one-way anova

Cria e salva os Box plot

In [0]:
boxPlot(data)

Cria e salva os demais Plots

In [0]:
plot_to_files(data)

Encerra o arquivo com o resultado dos calculos estatísticos 

In [0]:
outputfile.close()

Cria um relatório interativo da estatística da base de dados utilizando o Pandas Profiling 

In [0]:
profile = ProfileReport(data, title='Relatório - Pandas Profiling', html={'style':{'full_width':True}})
profile.to_file(output_file=(path + "/results/PandasProfiling.html"))

**Programa encerrado.**

Verifique a pasta 'results' para analisar seus resultados.

Obrigado por utilizar o script e não se esqueça de nos mencionar em seu artigo.